In [1]:
import collections
import networkx as nx
import sys
import os
sys.path.append(r'c:\\Users\\rotim\\OneDrive\\Documents\\Reading\\graph-code\\Skoltech-PhD-Thesis\\_codes\\SIR\\Influential-nodes-identification-in-complex-networks-via-information-entropy')
from algorithms import *
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle

In [9]:
graph_collections, graph_name = read_graph("./test-graphs")


{'ABILENE': {'index': 0, 'nodes': 11, 'edges': 14}, 'ABVT': {'index': 1, 'nodes': 23, 'edges': 31}, 'AGIS': {'index': 2, 'nodes': 25, 'edges': 30}, 'ARN.GRAPHML': {'index': 3, 'nodes': 30, 'edges': 29}, 'BELLCANADA': {'index': 4, 'nodes': 48, 'edges': 64}, 'BELLSOUTH.GRAPHML': {'index': 5, 'nodes': 51, 'edges': 66}, 'BICS.GRAPHML': {'index': 6, 'nodes': 33, 'edges': 48}, 'CERNET.GRAPHML': {'index': 7, 'nodes': 41, 'edges': 58}, 'CESNET200706.GRAPHML': {'index': 8, 'nodes': 44, 'edges': 51}, 'CHINANET.GRAPHML': {'index': 9, 'nodes': 42, 'edges': 66}, 'COGENTCO.GRAPHML': {'index': 10, 'nodes': 197, 'edges': 243}, 'COLT.GRAPHML': {'index': 11, 'nodes': 153, 'edges': 177}, 'DELTACOM.GRAPHML': {'index': 12, 'nodes': 113, 'edges': 161}, 'GARR201110.GRAPHML': {'index': 13, 'nodes': 59, 'edges': 74}, 'GEANT2001': {'index': 14, 'nodes': 27, 'edges': 38}, 'GRNET.GRAPHML': {'index': 15, 'nodes': 37, 'edges': 42}, 'INTELLIFIBER.GRAPHML': {'index': 16, 'nodes': 73, 'edges': 95}, 'INTEROUTE.GRAPHML'

In [23]:
G = graph_collections[0]
# geo_loc_data = get_geo_data("Internet2LatLong.csv")
geo_loc_data = {k: {"lat": v.get('Latitude', 0), "long": v.get('Longitude', 0)}
                for (k, v) in G.nodes(data=True)}
# print(geo_loc_data)
edge_geo_data_combined = assign_location(G, geo_loc_data)
# set edge attributes
set_edge_attr(G, edge_geo_data_combined)


In [35]:
node_strength_2_deg = [(i, sum(hub_information(G, i, 1)))for (i, j) in sorted(list(G.degree()), key=lambda item: int(item[0]))]


In [39]:

source_1, source_2, source_3, source_4, *others = closeness_centrality(G), eccentricity(G), betweenness_centrality(G, weight="weight"), node_strength_2_deg, load_centrality(
    G), current_flow_betweenness_centrality(G), approximate_current_flow_betweenness_centrality(G)  # eigenvector_centrality(G),

print(source_2, source_3)


[('0', 5), ('2', 5), ('3', 5), ('4', 5), ('1', 4), ('5', 4), ('6', 4), ('9', 4), ('7', 3), ('8', 3), ('10', 3)] [('7', 0.4666666666666667), ('10', 0.4222222222222222), ('6', 0.35555555555555557), ('9', 0.2), ('1', 0.11111111111111112), ('4', 0.11111111111111112), ('2', 0.06666666666666667), ('8', 0.06666666666666667), ('0', 0.022222222222222223), ('5', 0.022222222222222223), ('3', 0.0)]


In [40]:
maxi_mini_result = maxi_mini(source_1, source_2, source_3, source_4,no_of_evidences=4)
probability_weights_multi_res=probability_weights_multi(maxi_mini_result, e_1=source_1, e_2=source_2, e_3=source_3, e_4=source_4, number_of_evidences=4)
convert_dict_multi_results = convert_dict_multi(probability_weights_multi_res)
rank_result_multi(convert_dict_multi_results)


[('9', 1.0771438850229316),
 ('2', 0.8701289629706194),
 ('1', 0.7199198729496399),
 ('6', 0.6262826205553464),
 ('7', 0.39102175741555845)]